# Predicting Bike Rentals

## Project

Imagine how important it is for a bike-sharing company to know how many bikes it needs. If there are not enough of them, it loses potential clients. If there are more than are actually needed, they just stay and generate costs. I'm going to help solve this problem by predicting these numbers.

I will work on a real data collected by such a company. I am going to use a neural network and code it myself as this will be a simple network with one hidden layer. At last I will tune the hyperparameters and compare the results.

## Import resources

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Load the data

I'm going to work with data collected every hour. Let's load it and see its structure. 

In [2]:
rides = pd.read_csv('data/hour.csv')
rides.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


As you can see there are a few types of information. There is information whether a day is a working day, weekend or holiday. Bike usage depends on the weather and there are a few variables measuring the weather conditions. Finally, there is the number of rentals on a given hour. I am interested in a total count `cnt` without distinction between casual and registered rides.